Dataset for Tennis Court Keypoints

In [ ]:
# In order to extract the data set use this particular link of github

# https://github.com/yastrebksv/TennisCourtDetector.git

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import json
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess


TARGET_SIZE = (224, 224)
NUM_KEYPOINTS = 14 # 7 keypoints * 2 coordinates (x, y)

class KeypointsSequence(Sequence):

    def __init__(self, img_dir, data_file, batch_size, shuffle=True):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end() # Initial shuffle
    
    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size))
    
    def on_epoch_end(self):
        """Shuffle indices for the next epoch, matching PyTorch's shuffle=True"""
        self.indices = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        """Generate one batch of data"""
        
        # Get the indices for the current batch
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        
        # Initialize batch arrays
        X_batch = np.zeros((self.batch_size, *TARGET_SIZE, 3), dtype=np.float32)
        Y_batch = np.zeros((self.batch_size, NUM_KEYPOINTS * 2), dtype=np.float32)
        
        for i, data_idx in enumerate(batch_indices):
            item = self.data[data_idx]
            
            # --- Image Loading and Resizing (Matching cv2/transforms) ---
            img_path = f"{self.img_dir}/{item['id']}.png"
            img = cv2.imread(img_path)
            h, w = img.shape[:2]
            
            # Convert BGR to RGB
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Resize image to target size
            img_resized = cv2.resize(img_rgb, TARGET_SIZE, interpolation=cv2.INTER_LINEAR)
            
            # Preprocess for ResNet (Normalization/Standardization equivalent to PyTorch's)
            # The Keras `preprocess_input` for ResNet handles the necessary normalization
            img_preprocessed = resnet_preprocess(img_resized.astype(np.float32))
            
            X_batch[i,] = img_preprocessed
            
            # --- Keypoint Handling (Matching kps scaling) ---
            kps = np.array(item['kps']).flatten().astype(np.float32)
            
            # Apply same scaling logic as PyTorch code:
            # kps[::2] *= 224.0 / w   # Adjust x coordinates
            # kps[1::2] *= 224.0 / h # Adjust y coordinates
            kps[::2] = kps[::2] * (TARGET_SIZE[0] / w) # X
            kps[1::2] = kps[1::2] * (TARGET_SIZE[1] / h) # Y
            
            Y_batch[i,] = kps
            
        return X_batch, Y_batch

# Instantiate DataLoaders (Sequences)
BATCH_SIZE = 8 # Same as PyTorch code
train_sequence = KeypointsSequence("data/images", "data/data_train.json", BATCH_SIZE, shuffle=True)
val_sequence = KeypointsSequence("data/images", "data/data_val.json", BATCH_SIZE, shuffle=True)